In [ ]:
import polars as pl
import polars_ds as pds

In [ ]:
df = pds.random_data(size=5_000, n_cols = 0).select(
    pds.random(0.0, 1.0).alias("x1"),
    pds.random(0.0, 1.0).alias("x2"),
    pds.random(0.0, 1.0).alias("x3"),
)

In [ ]:
df.select(
    pds.query_transfer_entropy("x1", "x2", k = 3)
).item(0,0)

In [ ]:

df = pl.DataFrame({
    "id": [0,1,2,3],
    "a1": [0.1, 0.2, 0.3, 0.4],
    "a2": [0.1, 0.2, 0.3, 0.4],
    "a3": [0.1, 0.2, 0.3, 0.4],
    "can_eval": [1, 0, 1, 1]
})

In [ ]:
temp = df.with_columns(

    pds.query_knn_ptwise(
        "a1", "a2", "a3",
        index = "id",  # pl.col("id"), str | pl.Expr
        k = 1, 
        dist = "l2", # squared l2
        parallel = False,
        eval_mask = pl.col("can_eval") == 1,
        data_mask = pl.col("can_eval") == 1,
        # data_mask = pl.col("can_select") == 1
    ).alias("best friends")

)
temp

In [ ]:
list(temp["best friends"])[1] is None

In [ ]:
df.select(
    "id",
    "can_select",
    pds.query_knn_ptwise(
        pl.col("var1"), pl.col("var2"), pl.col("var3"), # Columns used as the coordinates in n-d space
        index = "id",  # pl.col("id"), str | pl.Expr
        k = 3, 
        dist = "l2", # squared l2
        parallel = True,
        eval_mask = pl.col("can_select") == 1,
        # data_mask = pl.col("can_select") == 1
    ).alias("best friends")

)

# .filter(
#     pl.col("can_select") == 1
# )

In [ ]:
df_test = df.lazy().select(
    "id",
    "can_select",
    pl.when(
        pl.col("can_select") == 1
    ).then(
        pds.query_knn_ptwise(
            *(pl.col(f"var{i}") for i in range(2)), # Columns used as the coordinates in n-d space
            index = "id",  # pl.col("id"), str | pl.Expr
            k = 5000, 
            dist = "l2", # squared l2
            parallel = True
        )
    ).otherwise(
        None
    )    
    .alias("best friends")
).collect()

In [ ]:
df_test

In [ ]:
df_test.filter(
    pl.col("can_select") == 1
)

In [ ]:
pl.show_versions()